In [6]:
'''
#Example script to generate text from Nietzsche's writings.

At least 20 epochs are required before the generated text
starts sounding coherent.

It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

In [7]:
#path = get_file(
#    'nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
path = "georgt-cl.txt"

In [8]:
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

def clean_doc(doc):
    # replace '--' with a space ' '
    doc = doc.replace('--', ' ')
    doc = doc.replace('*', ' ')
    return doc

text = clean_doc(text)

corpus length: 106327


In [9]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))

print(chars)

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1



total chars: 48
['\n', ' ', '!', "'", ',', '-', '.', '1', '2', '3', '4', '5', '6', ':', ';', '?', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ß', 'ä', 'ö', 'ü', '\ufeff']
nb sequences: 35422
Vectorization...


In [10]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)       

Build model...
Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

model.save("takl.h5")

Epoch 1/60
35422/35422 [==============================] - 12s 329us/step - loss: 1.5988

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "sie mit milch getüncht haben.
  der wahn"
sie mit milch getüncht haben.
  der wahnsin schlangen schauken der schaupern der schwarzen herzstern der schaupten der schauken der schauken der schauken der schaupern der stille der schauken des haups.
  der schweigen der schauken der ansen der schauken der schaupen der schauken der schaupen der schweigen.
  der schweigende stille des schauken leise stille der schauken der schweigenden herzen haupt.
  der schweigende im schauken der sc
----- diversity: 0.5
----- Generating with seed: "sie mit milch getüncht haben.
  der wahn"
sie mit milch getüncht haben.
  der wahnsich ein hauten stille kahl und schwarzen stille der haupt.
  die schwarzen kanz und schauken der todener kahlen.
  grühlen schwarzen wold
ein schrolke des mänden gestalt.
  im grau verfüllt der es und der tri

  purpurn finnern pfader betügt in silber
  des kommer
  wie sigen im den kahr, nüpt im schwarzen auf rins,
  beschlitt ein frommen warsches wald,
  ergrunde der erglichen ein fratter
  rakahn lächt der schwarze tränten strür uns resten,
  ein gerbarter karten höll in seinern purpurn gaskatig, das leibe balp, des früpling,
----- diversity: 1.2
----- Generating with seed: "mäher nachmittags geschnitten.
  geduldi"
mäher nachmittags geschnitten.
  geduldig schrit
  aus asteim,
  d.

  it gesülbt des torüchftrauen, beis
  herz von oklache; voel
l ichtes , elal, verlägratt in früht der schläse,
  vom mot die kraupfein,
  einig ein rotag as rotirc vordölten ha. ahr tod lich.;
  das herbstunsting terrunk.




korb
  und untall süßer sie und sundu,
  sonne
  enhst unter stunde. niernenh.
  fremde sanftes gorgener mohr,
  wenn gödüs, goomel. sundendem o
Epoch 5/60
35422/35422 [==============================] - 12s 331us/step - loss: 1.2219

----- Generating text after Epoch: 4
----- diversity:

  schmerz,                                                                                                                                                                                                                                
----- diversity: 0.5
----- Generating with seed: "ch eine
goldene wolke auf. bei der mühle"
ch eine
goldene wolke auf. bei der mühlern auf des wandern sterne.
  das hirtern kahl.
  purpurner kühle des stirne goldene herbstliche schrei
  und leibe der stille ein mondenen sterne.

  rot schmürz und scheint der weihe des haupt:
  die schmerz, wo stille stille des herbstliche sterne.




hellerdes schrecken vor sehn
  und frieden schwers, die herzsige die wirnern haupt.

  verfallene stadter und das hügel.




die auchmen


  ansl
----- diversity: 1.0
----- Generating with seed: "ch eine
goldene wolke auf. bei der mühle"
ch eine
goldene wolke auf. bei der mühle und taken.

  verfillener roor alter schritt


  der schlesphere kange die schwerze nächte des mun;

frei ergrünt der bach, wo silbernen wind.
  ooger zumper, ;

  intersphegelten höntee,
  und vörbe
schelbern blitzt dich ihr schwarz und wlita
  gehanschlachen
  in storne auf ofäten die lege die bruderfaub,
  !
inter hörber des munabeflusch ein gebestark. sienersaküstlicht gewanden.
  sanken,  in gräutregesunken, purpurn
  über die kühle dort und fönsten reite.
  besklden klage der mörlen härme,
  der spräch stürbenes hänschen v
Epoch 12/60
35422/35422 [==============================] - 11s 324us/step - loss: 0.9529

----- Generating text after Epoch: 11
----- diversity: 0.2
----- Generating with seed: "reuz und abend;
  umfängt den tönenden m"
reuz und abend;
  umfängt den tönenden mirchen ange schweigen.

  o win gestunge wild,
  das ihr herz versternen angeg der toten.

  reiner abends in der steinernen friedhen räuten.
  dunkle klage der fluchter schweigender grauen,
  es ist der abends in die bange schweigen.
  ein herzen rot der toten augenbauten.
  der abends hingen schweigende

  die stirne im wald der toten baum und gottie hinschlacht,
  die fälle in kroll.
  fall der toten baum, des verstorbenen antlitz
  des hand ich, daß er schweigende schweigen.
  wo die klingt ihr rauchen und das haupt im dorf. ein hirt im dunkel.
----- diversity: 1.0
----- Generating with seed: "torben breitet  
  und schatten sind um "
torben breitet  
  und schatten sind um schatten lahn.

  mit stille immersärber das dämmerberreurk,
  die müldar ihm holi,, dunklem fing und über sind,
  der kühle des zug den wald ind des weißen luit und grau;
  o win tonz, die milbernen blauer legende hinstarz hert ein.
  lu kliche wie
schaste tritz
  viellauf vor ein dunkel gegt dunkel wassel föhfen.

  öfen dunkler trauer,  asflembung,
  die seine ;
  assorpflum die vierotfedische 
----- diversity: 1.2
----- Generating with seed: "torben breitet  
  und schatten sind um "
torben breitet  
  und schatten sind um aum die eitennen wallenn fahle.
  o mwin grühliches mohrtheren.

  an verfallen aflen pf

  aus der wunde unter der fleden.
  ein herbstlich ich dunkler schatten der toten augen
  und die heißen augen im schatten der stemmen.
  ein herbstlich ich dunkler breisen hinsank,
  die herbstliche stegen hinabstein.

  im hummel schatten der flammender stille,
  wander geberkleichtet die schwester. stille des todes, wander der toten baum;
  die kristalle ergeln die klage der toten augen.
  im haumverruch die grünen 
----- diversity: 0.5
----- Generating with seed: "ine legende ging,
  aus der wunde unter "
ine legende ging,
  aus der wunde unter der kirchen
eines gols aus herbstlicher pfade
  in die nacht im haum rings
  der fledende himmel ist ein harren von verfall.
  der troms tene hinabstillen schwarze herbst die flamme
fallen träne wie bleichen die herbst ind schatten des engels.
  ein kist due lestheites himmel tritt anj es grünen sternen
  und die wange sinkt im kahlen wahrden hin.
  im haum, die koten dämmernden himmel
  und die f
----- diversity: 1.0
----- Generating with s





ne
angolln einer rufen frauen.
  im hagendim müßer auverdlächten
  am a
----- diversity: 1.2
----- Generating with seed: "mmel der zeit;
  über ein träumendes nei"
mmel der zeit;
  über ein träumendes neigt; auf roter nachtätte beugt im dumpuftern
  gehallwarr krhist über die
augen.

  mitter saucht bricht einer stepternübättentappätt im grauungstagd, das anssalt
  von weedes dei in augen.

  o wint,
  wald, das der dunkle wied ihr braunem rauern,
  aber ovem hummel und.
  die hieder vom grau und wold
  ist einer vom haupt ih schwarz michen der mond.


wande legrte fahl des abends blauen zügervero
Epoch 23/60
35422/35422 [==============================] - 12s 331us/step - loss: 0.8149

----- Generating text after Epoch: 22
----- diversity: 0.2
----- Generating with seed: "rgang


seltsam sind die nächtigen pfade"
rgang


seltsam sind die nächtigen pfaden
  der stiogeln des wandstern hends, da ich deine leden das leise seiner leise
  in der schweigend stille des wanden,
  die sterne

                                                                                                                                                                                                                                                                                                                                                                                                         
----- diversity: 1.0
----- Generating with seed: "erlen hin.

  in gärten sinken glocken l"
erlen hin.

  in gärten sinken glocken ling die löserbegen,
  wenter seele kom sommalen sanftes meer sie weiger.
  ein hunmverrumpfe in singen durchzbege;
  wan blunken herzern loder brauer; gott das hin. auf dheser luchen.
  goldner gebel und basten kran.
  wachringelt das kleischende braue.
  des walds, von wees, das verfieke und lem herbich
  und herz vom grauen gestalt verbösen.
  ruft die weib, under schluß und leise kinsen.
  das 
----- diversity: 1.2
----- Generating with seed: "erlen hin.

  in gärten

  umfirgend früster sindende gestalt erfuflen vöte den greisen dor
Epoch 30/60
35422/35422 [==============================] - 12s 330us/step - loss: 0.7515

----- Generating text after Epoch: 29
----- diversity: 0.2
----- Generating with seed: "und ich fuhr auf gebogenem kahn über den"
und ich fuhr auf gebogenem kahn über den baumen,
  und der schlaf die kühlem sauf und licht.
in am abend ein rosigen legenden himmel und es is ihrem ahog.
  ein kind ich sich das ein sonkten augen
  hingramme des heimalung,
  die fönenden mauer bivern, die fruckt der schlaf.

  o win mohe schatten des herbste des knaben des herbstes
  und des hags dämmerung
  und die stirne mog der schläfe des herbste,
  der schwestermauer  schrei
  des
----- diversity: 0.5
----- Generating with seed: "und ich fuhr auf gebogenem kahn über den"
und ich fuhr auf gebogenem kahn über den parken geschlechts. o wilden kirchen händ in die sternenhmulin,
  erstorbene glächt ein betitten augen voll,
  vor danft die schläfen des w

  gelwückt verschuin.

  und die stirne am zweißen und einner haupt, baumt der schweigenden alten der frau'n.
  in kähreckt sich die sonne im schwarzen vogel unter
  am abend erschieinernabgewind oht und es er kaömt
  der dunkle immer bnau hos.

  voll die bösen und fröt in ihr grau im zaumndwind
  ein hvers, und silbernen rorrummer
  blinde mundche brinnt in schwarzen gitten; und fal
----- diversity: 1.2
----- Generating with seed: "
  eisige winde im dunkel greinen.




d"

  eisige winde im dunkel greinen.




der trauer




stindallnumei


  mübter die bhaden od
eiten und ich raten warf kröm af den tröubötschligt die gehbung.

umrei


  man verwelkein braunen
  bnin; ferner kindhend frutter.
  vom hveten mag wern und föht in der kirb,
  der aus die stirne mohr auf vom wald.

  über dem schluchts, das dunkle blumen sternen,
  plaug vergolk
  viere kühlete; og
  grank nerten leise raat die ansel saugen
  
Epoch 34/60
35422/35422 [==============================] - 12s 329us/step - los